In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import sys, gc, os
sys.path.append('/home/jovyan/work/phd/')

/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [ ]:
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE79670
os.system("wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE79nnn/GSE79670/matrix/GSE79670_series_matrix.txt.gz")
os.system("gunzip GSE79670_series_matrix.txt.gz")
os.system("wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE79nnn/GSE79668/suppl/GSE79668%5F51%5Ftumors%5Fsharedgenecounts%2Etxt%2Egz")

In [ ]:
os.system("wget https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE79668&format=file&file=GSE79668%5F51%5Ftumors%5Fsharedgenecounts%2Etxt%2Egz")

In [ ]:
df = pd.read_csv("GSE79668_51_tumors_sharedgenecounts.txt.gz", sep="\t")
df.index=["ENSG"+g.split("ENSG")[1].split(".")[0] for g in df["Gene_EnsembleID"]]
df.drop("Gene_EnsembleID",1,inplace=True)
df.to_csv("mainTable_all.csv", index=True, header=True)

In [ ]:
with open("GSE79670_series_matrix.txt", "r") as file:
    data=[row[:-1].replace('"',"").split("\t") for row in file.readlines()]

In [ ]:
df_data = pd.DataFrame(data=data[24:]).transpose()
new_col = [col[1:] for col in df_data.loc[0,:]]
new_col[9:18]=[f"Sample_characteristics_ch{i}" for i in range(9)]
df_data.columns=new_col
df_data.drop(0,0,inplace=True)
df_data.set_index("Sample_title", inplace=True)
df_data.index = [sample[:13] for sample in df_data.index]
df_data.head()

In [ ]:
df.columns.isin(df_data.index).astype(int).sum()

In [ ]:
adata = sc.AnnData(X=df.transpose(), obs=df_files.reindex(index=df.columns))
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000)
sc.pl.highly_variable_genes(adata)
hvg = adata.var[adata.var["highly_variable"]==True].index

In [ ]:
df.reindex(index=hvg).to_csv("mainTable.csv", index=True, header=True)

In [ ]:
df_files = df_data
df_files.fillna("unknown").to_csv("files.dat")

In [ ]:
os.system("python3.6 make_graph.py")

In [2]:
df_files=pd.read_csv("files.dat", index_col=0)

In [32]:
bins=np.append([-2,2],np.linspace(30,3170,4))
df_files["Survival(days)"]=pd.cut([int(t.split(": ")[1]) if len(t) <= 30 else -1 for t in df_files["Sample_characteristics_ch1"]], bins=bins, labels=bins[1:])

In [34]:
df_files.to_csv("files.dat")

In [33]:
df_files["Survival(days)"]

T_07_07_A082a    1076.666667
T_06_01_A033a    1076.666667
T_06_11_A168a    1076.666667
T_06_04_A296a    1076.666667
T_07_11_A090a    1076.666667
                    ...     
AsPC_1_SL2329       2.000000
Capan_2_SL232       2.000000
Capan_1_SL232       2.000000
HPAF_2_SL2329       2.000000
Panc_2_03_SL2       2.000000
Name: Survival(days), Length: 65, dtype: category
Categories (5, float64): [2.000000 < 30.000000 < 1076.666667 < 2123.333333 < 3170.000000]